In [1]:
import ee
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1wGpIrDcznlngmK87Bj1iB2dNRyCHisnkQZrBZV7GcWLsOg4_4dDnMc

Successfully saved authorization token.


In [2]:
# Import the Image function from the IPython.display module. 
from IPython.display import Image

### Simple way to get image

In [3]:
image = ee.Image('srtm90_v4')
print(image.getInfo())
Image(url = image.getThumbURL({'min': 0, 'max': 4000, 'dimensions': 512,
                'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}))

{'type': 'Image', 'bands': [{'id': 'elevation', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [432000, 144000], 'crs': 'EPSG:4326', 'crs_transform': [0.000833333333333, 0, -180, 0, -0.000833333333333, 60]}], 'version': 1494271934303000.0, 'id': 'srtm90_v4', 'properties': {'system:time_start': 950227200000, 'system:time_end': 951177600000, 'system:asset_size': 18827626666}}


In [4]:
# Get a download URL for an image.
image1 = ee.Image('srtm90_v4')
path = image1.getDownloadUrl({
    'scale': 30,
    'crs': 'EPSG:4326',
    'region': '[[-120, 35], [-119, 35], [-119, 34], [-120, 34]]'
})
print(path)

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/6372828bf91c925d155cb0ea8c792d61-302e5c60f84d77bd6e0762ac166550b5:getPixels


## Using thirdparty packages

In [5]:
import folium
import geehydro
from datetime import datetime as dt

In [6]:
# the Ituna/Itatá Indigenous Land in Brazil.
Ituna_map = folium.Map(location=[-4.06738, -52.034], zoom_start=10)
# Ituna_map

In [7]:
# get data from landsat-8
landsat = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")
# setting the Area of Interest (AOI)
Ituna_AOI = ee.Geometry.Rectangle([-51.84448, -3.92180,
                                   -52.23999, -4.38201])
# filter area
landsat_AOI = landsat.filterBounds(Ituna_AOI)


# choose dates
# landsat = landsat.filterDate('2019-07-01','2019-12-01')
# landsat_AOI.getInfo()

# choose image
# the least cloudy image
least_cloudy = ee.Image(landsat_AOI.sort('CLOUD_COVER').first())
# how cloudy is it?
print('Cloud Cover (%):', least_cloudy.get('CLOUD_COVER').getInfo())

Cloud Cover (%): 0


In [8]:
# visualizing satellite imagery
# setting the Area of Interest (AOI)
# Ituna_AOI = ee.Geometry.Rectangle([-51.84448, -3.92180,
#                                    -52.23999, -4.38201])
parameters = {'min': 0,
              'max': 1000,
              'dimensions': 512,
              'bands': ['B4', 'B3', 'B2'],
              'region': Ituna_AOI}
             
Image(url = least_cloudy.getThumbUrl(parameters))

## Image collections

**Landsat-8**

In [9]:
# get data from landsat-8
landsat = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")
# setting the Area of Interest (AOI)
Ituna_AOI = ee.Geometry.Rectangle([137.3, 34.65, 137.5, 34.85])
# Ituna_AOI = ee.Geometry.Rectangle([137.325, 34.65, 137.35, 34.7])

# filter area
landsat_AOI = landsat.filterBounds(Ituna_AOI)

least_cloudy = ee.Image(landsat_AOI.sort('CLOUD_COVER').first())

parameters = {'min': 0,
              'max': 3000,
              'dimensions': 1024*3,
              'bands': ['B4', 'B3', 'B2'],
              'region': Ituna_AOI}
             
Image(url = least_cloudy.getThumbUrl(parameters))

**Sentinel-2 MSI: MultiSpectral Instrument, Level-2A**

In [10]:
def maskS2clouds(image):
    """
     Function to mask clouds using the Sentinel-2 QA band
     @param {ee.Image} image Sentinel-2 image
     @return {ee.Image} cloud masked Sentinel-2 image
     """
    qa = image.select('QA60');

    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = (1 << 10)
    cirrusBitMask = (1 << 11)

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0) and (qa.bitwiseAnd(cirrusBitMask).eq(0))

    return image.updateMask(mask).divide(10000)

In [18]:
landsat = ee.ImageCollection("COPERNICUS/S2_SR")\
            .filterDate('2018-01-01', '2018-12-31')\
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20))\
            .map(maskS2clouds)
# setting the Area of Interest (AOI)
# Ituna_AOI = ee.Geometry.Rectangle([137.3, 34.65, 137.5, 34.85])
# Ituna_AOI = ee.Geometry.Rectangle([137.325, 34.65, 137.35, 34.7])
# Ituna_AOI = ee.Geometry.Rectangle([137.325, 34.65, 137.35, 34.675])
# Ituna_AOI = ee.Geometry.Rectangle([137.325, 34.65, 137.33, 34.655])
Ituna_AOI = ee.Geometry.Rectangle([137.3, 34.65, 137.325, 34.675])

# filter area
landsat_AOI = landsat.filterBounds(Ituna_AOI)

least_cloudy = ee.Image(landsat_AOI.sort('LAND_COVER').first())

parameters = {'min': 0,
              'max': 0.3,
              # 'dimensions': 1024*3,
              'scale':1,
              'bands': ['B4', 'B3', 'B2'],
              'region': Ituna_AOI}
             
Image(url = least_cloudy.getThumbUrl(parameters))

In [26]:
# export image
color_image = landsat_AOI.select(['B4', 'B3', 'B2'])
color_image = ee.Image(landsat_AOI.sort('LAND_COVER').first())
task_config = {
    # 'description': 'imageToDriveExample',
    'scale': 5,
    'region': Ituna_AOI, #["coordinates"][0]
    'folder':'map_exports',
    }
task = ee.batch.Export.image(color_image, 'toyohashi_export_image', task_config)
task.start()

In [30]:
# another export format
landsat = ee.ImageCollection("COPERNICUS/S2_SR")\
            .filterDate('2018-01-01', '2018-12-31')\
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',5))\
            .map(maskS2clouds).select(['B4', 'B3', 'B2'])
# setting the Area of Interest (AOI)
# Ituna_AOI = ee.Geometry.Rectangle([137.3, 34.65, 137.5, 34.85])
# Ituna_AOI = ee.Geometry.Rectangle([137.325, 34.65, 137.35, 34.7])
# Ituna_AOI = ee.Geometry.Rectangle([137.325, 34.65, 137.35, 34.675])
# Ituna_AOI = ee.Geometry.Rectangle([137.325, 34.65, 137.33, 34.655])
Ituna_AOI = ee.Geometry.Rectangle([137.3, 34.65, 137.325, 34.675])

# filter area
landsat_AOI = landsat.filterBounds(Ituna_AOI)

least_cloudy = ee.Image(landsat_AOI.sort('LAND_COVER').first())
task_config = {
    'scale': 1,
    'region': Ituna_AOI,
    'folder':'map_exports',
    }
task = ee.batch.Export.image(least_cloudy, 'toyohashi_export_full_scale', task_config)
task.start()

### NAIP: National Agriculture Imagery Program (has only US data)

In [36]:
import io
from PIL import Image as pil_Image

### Sentinel-2 MSI: MultiSpectral Instrument, Level-1C

In [ ]:
landsat = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")
# setting the Area of Interest
toyohashi = ee.Geometry.Rectangle([137.3, 32.65, 137.5, 34.85])

# filter area
landsat_toyo = landsat.filterBounds(toyohashi)


# choose dates
# landsat = landsat.filterDate('2019-07-01','2019-12-01')
landsat_toyo = landsat_toyo.filterDate('2016-01-01', '2016-12-31')
# print(landsat_AOI.getInfo())

# choose image
# the least cloudy image
land_cover_toyo = ee.Image(landsat_toyo)
print(type(land_cover_toyo))

In [ ]:
# Display a thumbnail of global elevation.
Image(url = land_cover_toyo.updateMask(land_cover_toyo.gt(1))
  .getThumbURL({'min': 0, 'max': 4000, 'dimensions': 512,
                'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}))

In [ ]:

# Import the matplotlib.pyplot module.
import matplotlib.pyplot as plt

# Fetch a Landsat image.
img = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_034033_20000913')

# Select Red and NIR bands, scale them, and sample 500 points.
samp_fc = img.select(['B3','B4']).divide(10000).sample(scale=30, numPixels=500)

# Arrange the sample as a list of lists.
samp_dict = samp_fc.reduceColumns(ee.Reducer.toList().repeat(2), ['B3', 'B4'])
samp_list = ee.List(samp_dict.get('list'))

# Save server-side ee.List as a client-side Python list.
samp_data = samp_list.getInfo()

# Display a scatter plot of Red-NIR sample pairs using matplotlib.
plt.scatter(samp_data[0], samp_data[1], alpha=0.2)
plt.xlabel('Red', fontsize=12)
plt.ylabel('NIR', fontsize=12)
plt.show()